#### [ 이미지를 파일 데이터셋으로 생성 ]
- 이미지 파일의 raw data 사용
- 데이터셋 파일 : csv, json, ... 등등 개발자 선택
- 데이터셋 파일 구성 : 라벨 +피쳐,  피쳐+라벨
- 1개의 행(row)에 1개 이미지가 저장됨!

[1] 모듈 로딩 및 데이터 준비 <hr>

In [3]:
## [1-1] 모듈 로딩
import cv2                        ## 일반적인 이미지 파일들에서 순수 이미지 데이터 다루는 모듈
import matplotlib.pyplot as plt   ## 시각화
import numpy as np                ## 순수 이미지 데이터의 저장 형식 ndarray
import os                         ## 폴더, 파일, 경로 관련 

In [4]:
## [1-2] 데이터 준비
IMG_DIR  = '../data/image/'              ## 이미지 존재 폴더
ROW, COL = 60, 60                        ## 이미지 리사이징 크기 
DS_DIR   = '../data/image/DS/'           ## 학습에 사용할 이미지 저장 폴더
IMG_CSV  = '../data/animal_color.csv'    ## 이미지 라벨과 이미지 로우데이터 저장 파일

DEBUG_MODE = True                        ## 개발 단계에서 코드 체크 용도

In [5]:
## 학습용 이미지 저장 폴더 생성
if not os.path.exists(DS_DIR):
    os.mkdir(DS_DIR)
else:
    print(f'{DS_DIR} 존재함!')

../data/image/DS/ 존재함!


In [9]:
## [1-3] 사용 이미지 파일리스트 추출
IMG_FILELIST = os.listdir(IMG_DIR)

for filename in IMG_FILELIST:
    if not os.path.isfile(IMG_DIR+filename):
        # 파일이 아닌 경우 처리 리스트에서 제거
        IMG_FILELIST.remove(filename)

if DEBUG_MODE: print(IMG_FILELIST)

['4027.png', '4star.jpg', '5shapes.jpg', 'abnormal.jpg', 'actor.jpg', 'aircraft.jpg', 'bad_rect.png', 'big_buck.avi', 'blank_500.jpg', 'boy_face.jpg', 'bright.jpg', 'bright_abnormal.jpg', 'cat.jpg', 'cat.png', 'cat_dog_gray', 'children.jpg', 'coins_connected.jpg', 'coins_spread1.jpg', 'cube.jpg', 'digits.png', 'drawing.jpg', 'dr_ochanomizu.jpg', 'figures.jpg', 'figures2.jpg', 'figures3.jpg', 'figures4.jpg', 'figures4.png', 'figures5.jpg', 'fish.jpg', 'flower.png', 'flower_v.jpg', 'flower_wall.jpg', 'full_body.jpg', 'gaussian_noise.jpg', 'girl.jpg', 'girl_face.jpg', 'girl_gray.jpg', 'gray_gradient.jpg', 'hand.jpg', 'highway.mp4', 'house.jpg', 'jetstar.jpg', 'lightning.png', 'lion_face.jpg', 'man_chromakey.jpg', 'man_face.jpg', 'mask_hannibal.png', 'model.jpg', 'model2.jpg', 'model3.jpg', 'moon_gray.jpg', 'morphological.png', 'morph_dot.png', 'morph_hole.png', 'motorbike.jpg', 'motorcycle.jpg', 'mountain.jpg', 'my_hand.jpg', 'opencv_logo.png', 'paper.jpg', 'pistol.jpg', 'pump_horse.jpg',

In [10]:
for filename in IMG_FILELIST:
    ## 파일명에서 확장자 추출
    if filename in ['.jpg','.jpeg','.png','.bmp']:
        pass
    else:
        IMG_FILELIST.remove(filename)


print(IMG_FILELIST)

['4star.jpg', 'abnormal.jpg', 'aircraft.jpg', 'big_buck.avi', 'boy_face.jpg', 'bright_abnormal.jpg', 'cat.png', 'children.jpg', 'coins_spread1.jpg', 'digits.png', 'dr_ochanomizu.jpg', 'figures2.jpg', 'figures4.jpg', 'figures5.jpg', 'flower.png', 'flower_wall.jpg', 'gaussian_noise.jpg', 'girl_face.jpg', 'gray_gradient.jpg', 'highway.mp4', 'jetstar.jpg', 'lion_face.jpg', 'man_face.jpg', 'model.jpg', 'model3.jpg', 'morphological.png', 'morph_hole.png', 'motorcycle.jpg', 'my_hand.jpg', 'paper.jpg', 'pump_horse.jpg', 'restaurant2.jpg', 'robot_arm2.jpg', 'salt_pepper_noise.jpg', 'shapes.png', 'shapes_donut.png', 'street.jpg', 'sudoku.png', 'taekwonv1.jpg', 'taekwonv3.jpg', 'walking.avi', 'yate.jpg']


[2] 이미지 데이터 로딩<hr>

In [8]:
## RGB Color 형식으로 이미지 데이터 로딩
with open(IMG_CSV, mode='a') as csvF:
    for filename in IMG_FILELIST:
        # - 3채널로 이미지 로딩
        rgbIMG = cv2.imread(IMG_DIR+filename, cv2.IMREAD_COLOR)
        print(f'rgbIMG => {rgbIMG.shape}')

        # - 사이즈 조정
        rgbIMG = cv2.resize(rgbIMG, (COL, ROW))
        print(f'rgbIMG => {rgbIMG.shape}')

        # - 파일로 저장 
        resizeFile = f'{DS_DIR}resize_{filename}'
        print(f'resizeFile => {resizeFile}')
        
        ## - opencV 채널 순서 BGR ==> 일반 RGB
        ret = cv2.imwrite(resizeFile, cv2.cvtColor(rgbIMG, cv2.COLOR_BGR2RGB))
        if not ret: print(f'Fail => {resizeFile}')

        ## - 채널 분리 
        b, g, r = cv2.split(rgbIMG)
        b, g, r = b.reshape(-1) , g.reshape(-1), r.reshape(-1)

        # # - CSV 파일 쓰기 => 라벨, 픽셀1, 픽셀2,....., 픽셀n
        label=filename[:3]
        data = np.concatenate((r, g, b)) ## 3채널 1개로 연결
        print(f'label => {label}, data => {data.shape}')

        sdata = list(map(lambda n: str(n), data)) 
        print(f'sdata => {sdata[:30]}')

        csvF.write(str(label)+",")                 # 숫자 라벨 쓰기
        csvF.write(','.join(sdata) + "\n")         # 리스트 이미지 데이터 -> 문자열 변환 후 쓰기
        

rgbIMG => (211, 210, 3)
rgbIMG => (60, 60, 3)
resizeFile => ../data/image/DS/resize_4star.jpg
label => 4st, data => (10800,)
sdata => ['255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255', '255']
rgbIMG => (360, 480, 3)
rgbIMG => (60, 60, 3)
resizeFile => ../data/image/DS/resize_abnormal.jpg
label => abn, data => (10800,)
sdata => ['152', '159', '118', '135', '122', '154', '160', '156', '149', '167', '173', '175', '175', '174', '174', '174', '175', '175', '175', '175', '175', '175', '175', '176', '176', '176', '176', '176', '176', '176']
rgbIMG => (451, 638, 3)
rgbIMG => (60, 60, 3)
resizeFile => ../data/image/DS/resize_aircraft.jpg
label => air, data => (10800,)
sdata => ['158', '158', '158', '158', '157', '158', '157', '157', '156', '157', '157', '156', '156', '155', '155', '154', '154', '154', '155', '154', '153', '153', '152', '153', '

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import pandas as pd

In [ ]:
colorDF = pd.read_csv(IMG_CSV, header=None)
colorDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Columns: 10801 entries, 0 to 10800
dtypes: int64(10800), object(1)
memory usage: 337.7+ KB


In [ ]:
colorDF.head(1)

,0,1,2,3,4,5,6,7,8,9,...,10791,10792,10793,10794,10795,10796,10797,10798,10799,10800
0,cat,177,178,181,182,184,186,187,188,189,...,106,241,255,255,225,172,153,221,207,207
